In [17]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(tableRt, tableType = "RESOURCE_TABLE")
setTable(table)

Set Redshift endpoint done. 2017-04-23T20:24:36


"beacons_4744"

In [18]:
# Packages
include("/data/notebook_home/Production/Lib/Include-Package-v1.0.jl")

#TV = timeVariables(2017,4,15,10,0,2017,4,15,10,9);
#TV = timeVariables(2017,4,21,10,0,2017,4,21,10,9);
TV = weeklyTimeVariables(days=7);

April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time
April 16, 2017 03:59:00 to April 23, 2017 03:59:00 UTC Time


In [19]:
include("/data/notebook_home/Production/Bodies/RD-Body-v1.0.jl")
customer = "Nat Geo" 
#productPageGroup = "%"
productPageGroup = "%" # primary page group
localUrl = "%"
resourceUrl = "%ng-black-logo.ngsversion.dafe07f7.png%"

localTable = "$(table)_Find_Resource_Details"
linesOutput = 25
minimumEncoded = 0
;

In [20]:
resourceMatched(tableRt;linesOut=3)
;

Matches For Url Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

count
597


In [21]:
resourceSummaryAllFields(tableRt;linesOut=3)
;

Raw Resource Url Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

params u,session id,session start,timestamp,url,initiator type,start time,redirect start,redirect end,fetch start,dns start,dns end,tcp connection start,secure connection start,tcp connection end,request start,response first byte,response last byte,worker start,encoded size,transferred size,decoded size,height,width,x,y
http://www.nationalgeographic.com/travel/destinations/europe/ireland/road-trip-history-ireland-ancient-east/?utm_source=Twitter&utm_medium=Social&utm_content=link_tw20170404travel-ireland&utm_campaign=Content&sf67898714=1,48c1d140-34c0-4009-b985-c24bbc2e10cc-ooi61k,"1,492,346,647,632",2017-04-16T12:44:18.280,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,img,468,0,0,468,0,0,0,0,0,496,514,549,NA,NA,NA,NA,NA,NA,NA,NA
http://news.nationalgeographic.com/2017/03/humpback-whales-swarms-south-africa/,6cd37d0f-536e-47e9-93c1-940d18e97cdc-oohxnx,"1,492,335,789,011",2017-04-16T09:47:05.427,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,img,693,0,0,693,0,0,0,0,0,699,707,716,NA,"23,967",0,NA,NA,NA,NA,NA
http://news.nationalgeographic.com/news/energy/2012/02/pictures/120228-spain-solar-energy/,ff79de5e-57a7-4c38-9e88-91af09c2835d-ooi0e4,"1,492,339,324,025",2017-04-16T10:42:39.684,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,img,"11,689",0,0,"11,689",0,0,0,0,0,"12,615","12,615","12,615",NA,NA,NA,NA,NA,NA,NA,NA


In [22]:
resourceSummary(tableRt;linesOut=linesOutput)
;

Resource Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

count,url
326,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
270,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
1,http://1.1.1.1/bmi/news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png


In [23]:
resourceSize(tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;

Size Details For Resource Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

count,encoded size,transferred size,decoded size,url
206,"23,967",0,NA,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
196,"23,967",0,NA,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
2,"4,601","4,974",NA,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
1,"4,601","5,006",NA,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
1,"23,967","24,209",NA,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
1,"6,074",0,NA,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png


Encoded Size Stats (Page Views are Groups In Above Table) April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

Page Views,Mean(ms),Median(ms),Min(ms),Max(ms),25 Percentile,75 Percentile
6,"14,530","15,020","4,601","23,967","4,969","23,967"


Transferred Size Stats April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

Page Views,Mean(ms),Median(ms),Min(ms),Max(ms),25 Percentile,75 Percentile
6,"5,698","2,487",0,"24,209",0,"4,998"


In [24]:
resourceScreen(tableRt;linesOut=linesOutput)
;

Screen Details For Resource Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

count,initiator type,height,width,x,y,url
326,img,NA,NA,NA,NA,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
270,img,NA,NA,NA,NA,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png
1,img,NA,NA,NA,NA,http://1.1.1.1/bmi/news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png


In [25]:
resourceSummaryDomainUrl(tableRt;linesOut=linesOutput)
;


Domain Url For Resource Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

count,url,params u
42,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/python-snake-swallows-man-whole-indonesia/
16,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/badger-buries-entire-cow-carcass/
14,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/photography/photo-of-the-day/2016/3/morskie-oko-scene/
12,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/02/crystal-caves-mine-microbes-mexico-boston-aaas-aliens-science/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170217news-cavemicrobes&utm_campaign=Content&sf57044048=1
12,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/photography/proof/2017/03/gelada-baboon-giving-birth/
12,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/magazine/2017/03/dark-star-deepest-cave-climbing-uzbekistan/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170207ngm-darkstar&utm_campaign=Content&sf54744609=1
11,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/channeled-scablands/
11,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/adventure/trips/best-trails/worlds-best-grail-trails/?utm_source=Facebook&utm_medium=Social&utm_content=link_fbadv20170323adv-besthikes&utm_campaign=Content&sf65421455=1
9,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/human-heart-spinach-leaf-medicine-science/
8,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/photography/proof/2017/03/gelada-baboon-giving-birth/?utm_source=NatGeocom&utm_medium=Email&utm_content=pom_20170326&utm_campaign=Content&utm_rd=1911474017


In [26]:
function resourceTime1(tableRt::ASCIIString;linesOut::Int64=25)
    
    try
        joinTables = query("""\
        select 
        count(*),
        avg(start_time) as "Start Time",
        avg(fetch_start) as "Fetch Start",
        avg(dns_end-dns_start) as "DNS ms",
        avg(tcp_connection_end-tcp_connection_start) as "TCP ms",
        avg(request_start) as "Req Start",
        avg(response_first_byte) as "Req FB",	
        avg(response_last_byte) as "Req LB",
        max(response_last_byte) as "Max Req LB",
        url,
        avg(redirect_start) as "Redirect Start",
        avg(redirect_end) as "Redirect End",
        avg(secure_connection_start) as "Secure Conn Start"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
          "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC)
        group by url
        order by count(*) desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceTime1 (generic function with 1 method)

In [27]:
resourceTime1(tableRt;linesOut=linesOutput)
;

Raw Resource Url Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

count,start time,fetch start,dns ms,tcp ms,req start,req fb,req lb,max req lb,url,redirect start,redirect end,secure conn start
326,"1,454","1,454",0,2,"1,481","1,922","2,172","57,951",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
270,"1,482","1,482",0,0,"1,501","1,840","1,986","82,097",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
1,"1,220","1,220",0,0,0,0,"1,282","1,282",http://1.1.1.1/bmi/news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0


In [28]:
function resourceTime2(tableRt::ASCIIString;linesOut::Int64=25)
    
    try
        timeTable = query("""\
        select 
        (response_last_byte-start_time) as "Time Taken",
        (start_time) as "Start Time",
        (fetch_start) as "Fetch Start",
        (dns_end-dns_start) as "DNS ms",
        (tcp_connection_end-tcp_connection_start) as "TCP ms",
        (request_start) as "Req Start",
        (response_first_byte) as "Req FB",	
        (response_last_byte) as "Req LB",
        url,
        (redirect_start) as "Redirect Start",
        (redirect_end) as "Redirect End",
        (secure_connection_start) as "Secure Conn Start"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
        "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC) and
        (response_last_byte-start_time) > 75 and (response_last_byte-start_time) < 10000
        order by "Time Taken" desc
        """);

        #todo remove negitives
        
        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        #scrubUrlToPrint(timeTable,limit=150)
        beautifyDF(timeTable[1:min(linesOut,end),:])
        
        timeTable = names!(timeTable[:,:],
        [symbol("taken"),symbol("start"),symbol("fetch"),symbol("dns"),symbol("tcp"),symbol("req_start"),symbol("req_fb"),symbol("req_lb"),symbol("url")
            ,symbol("redirect_start"),symbol("redirect_end"),symbol("secure_conn_start")])
        
        dv1 = timeTable[:taken]
        statsDF1 = limitedStatsFromDV(dv1)
        showLimitedStats(TV,statsDF1,"Time Taken Stats")
        
        dv2 = timeTable[:dns]
        statsDF2 = limitedStatsFromDV(dv2)
        showLimitedStats(TV,statsDF2,"DNS Stats")
        
        dv3 = timeTable[:tcp]
        statsDF3 = limitedStatsFromDV(dv3)
        showLimitedStats(TV,statsDF3,"TCP Stats")
        
        dv4 = timeTable[:start]
        statsDF4 = limitedStatsFromDV(dv4)
        showLimitedStats(TV,statsDF4,"Start Time On Page Stats")
        
        dv5 = timeTable[:fetch]
        statsDF5 = limitedStatsFromDV(dv5)
        showLimitedStats(TV,statsDF5,"Fetching Request Stats")
        
        dv6 = timeTable[:req_start]
        statsDF6 = limitedStatsFromDV(dv6)
        showLimitedStats(TV,statsDF6,"Request Start Stats")
        
        dv7 = timeTable[:req_fb]
        statsDF7 = limitedStatsFromDV(dv7)
        showLimitedStats(TV,statsDF7,"Request First Byte Stats")
        
        dv8 = timeTable[:req_lb]
        statsDF8 = limitedStatsFromDV(dv8)
        showLimitedStats(TV,statsDF8,"Request Last Byte Stats")
        
        
        
        
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceTime2 (generic function with 1 method)

In [29]:
resourceTime2(tableRt;linesOut=linesOutput)
;

Raw Resource Url Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

time taken,start time,fetch start,dns ms,tcp ms,req start,req fb,req lb,url,redirect start,redirect end,secure conn start
"8,506","6,404","6,404",0,0,"6,407","9,243","14,910",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"6,323","2,216","2,216",0,0,0,"5,182","8,539",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"6,302","2,225","2,225",0,0,0,"5,150","8,527",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"6,287","75,810","75,810",0,0,"76,653","81,633","82,097",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"5,347","2,467","2,467",0,0,0,"5,306","7,814",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"5,072","3,738","3,738",0,0,0,"6,728","8,810",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"5,033","4,358","4,358",0,0,"4,359","6,813","9,391",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"5,014","3,767","3,767",0,0,0,"6,694","8,781",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"4,569","2,738","2,738",0,0,0,"4,818","7,307",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0
"4,553","3,220","3,220",0,0,"7,454","7,644","7,773",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,0,0,0


Time Taken Stats April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

Page Views,Mean(ms),Median(ms),Min(ms),Max(ms),25 Percentile,75 Percentile
204,930,312,76,"8,506",122,835


DNS Stats April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

...

Page Views,Mean(ms),Median(ms),Min(ms),Max(ms),25 Percentile,75 Percentile
204,"2,197",595,0,"76,653",204,"1,536"


Request First Byte Stats April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

Page Views,Mean(ms),Median(ms),Min(ms),Max(ms),25 Percentile,75 Percentile
204,"2,949","1,127",141,"81,633",545,"3,034"


Request Last Byte Stats April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

Page Views,Mean(ms),Median(ms),Min(ms),Max(ms),25 Percentile,75 Percentile
204,"3,336","1,246",148,"82,097",651,"4,417"


In [30]:
function resourceTime3(tableRt::ASCIIString;linesOut::Int64=25)
    
    try
        joinTables = query("""\
        select 
        (response_last_byte-start_time) as "Time Taken",
        (start_time) as "Start Time",
        (fetch_start) as "Fetch S",
        (dns_end-dns_start) as "DNS ms",
        (tcp_connection_end-tcp_connection_start) as "TCP ms",
        (request_start) as "Req S",
        (response_first_byte) as "Req FB",	
        (response_last_byte) as "Req LB",
        url, params_u,
        (redirect_start) as "Redirect S",
        (redirect_end) as "Redirect E",
        (secure_connection_start) as "Secure Conn S"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
        "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC) and
        start_time > 10000
        order by start_time desc
        limit 25
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceTime3 (generic function with 1 method)

In [31]:
resourceTime3(tableRt;linesOut=linesOutput)
;

Raw Resource Url Pattern %ng-black-logo.ngsversion.dafe07f7.png% April 15, 2017 23:59:00 to April 22, 2017 23:59:00 Local Time

time taken,start time,fetch s,dns ms,tcp ms,req s,req fb,req lb,url,params u,redirect s,redirect e,secure conn s
"6,287","75,810","75,810",0,0,"76,653","81,633","82,097",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/environment/natural-disasters/volcanoes/,0,0,0
"15,490","26,152","26,152",0,0,"26,154","38,790","41,642",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/news/2015/02/150219-gold-hoard-coins-dinars-israel-fatimid-caesarea-archaeology/,0,0,0
14,"22,290","22,290",0,0,"22,301","22,303","22,304",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/animals/mammals/group/howler-monkeys/,0,0,0
"2,890","22,206","22,206",0,0,"25,096","25,096","25,096",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/monkeys-saved-lost-tourist-bolivian-amazon-shamans/,0,0,0
"21,561","20,243","20,243",0,0,"20,245","33,219","41,804",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/news/2012/02/120221-oldest-seeds-regenerated-plants-science/,0,0,0
6,"16,911","16,911",0,0,"16,912","16,914","16,917",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/01/scurvy-disease-discovery-jonathan-lamb/,0,0,0
130,"13,698","13,698",0,0,"13,761","13,761","13,828",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/channeled-scablands/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170328news-scablands2&utm_campaign=Content&sf66163662=1,0,0,0
624,"13,549","13,549",0,0,"13,910","14,142","14,173",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://www.nationalgeographic.com/travel/features/best-trips-2017/,0,0,0
555,"13,126","13,126",0,0,"13,681","13,681","13,681",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/monkeys-saved-lost-tourist-bolivian-amazon-shamans/,0,0,0
"2,322","12,537","12,537",0,0,"14,859","14,859","14,859",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.dafe07f7.png,http://news.nationalgeographic.com/2017/03/monkeys-saved-lost-tourist-bolivian-amazon-shamans/,0,0,0


In [32]:
## Resource and it parent is also interesting below this point

#try
#    query("""\
#        create or replace view $localTable as (
#            select * from $table 
#                where 
#                    "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC) and 
#                    page_group ilike '$(productPageGroup)' and
#                    params_u ilike '$(localUrl)' 
#        )
#    """)
#    cnt = query("""SELECT count(*) FROM $localTable""")
#    println("$localTable count is ",cnt[1,1])
#catch y
#    println("setupLocalTable Exception ",y)
#end
